# FM 9578 Assignment 2.1 - Week 3
Unit 1: Computational Finance -week 3.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# 5 years of daily prices for S&P/TSX
tsx = yf.download("^GSPTSE", start='2007-01-01', end='2012-01-01')
tsx

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-02,12924.500000,12989.599609,12908.400391,12923.700195,12923.700195,60857400
2007-01-03,12919.200195,12926.400391,12672.799805,12701.400391,12701.400391,199791400
2007-01-04,12679.200195,12701.400391,12519.400391,12553.599609,12553.599609,247371600
2007-01-05,12517.599609,12553.599609,12439.299805,12478.000000,12478.000000,201713900
2007-01-08,12558.299805,12617.299805,12478.000000,12553.099609,12553.099609,172708200
...,...,...,...,...,...,...
2011-12-22,11766.200195,11880.000000,11743.900391,11876.500000,11876.500000,172966700
2011-12-23,11900.599609,11949.099609,11884.200195,11926.700195,11926.700195,122283600
2011-12-28,11893.400391,11917.000000,11709.799805,11728.400391,11728.400391,98273900


In [2]:
# 1
# Date range of all weekdays 
weekdays = pd.date_range(start=tsx.index.min(), end=tsx.index.max(), freq='B')

# Reindex data to include all weekdays
tsx = tsx.reindex(weekdays)

# Fill missing values with previous day's price
tsx['Adj Close'] = tsx['Adj Close'].ffill()

tsx

,Open,High,Low,Close,Adj Close,Volume
2007-01-02,12924.500000,12989.599609,12908.400391,12923.700195,12923.700195,60857400.0
2007-01-03,12919.200195,12926.400391,12672.799805,12701.400391,12701.400391,199791400.0
2007-01-04,12679.200195,12701.400391,12519.400391,12553.599609,12553.599609,247371600.0
2007-01-05,12517.599609,12553.599609,12439.299805,12478.000000,12478.000000,201713900.0
2007-01-08,12558.299805,12617.299805,12478.000000,12553.099609,12553.099609,172708200.0
...,...,...,...,...,...,...
2011-12-26,NaN,NaN,NaN,NaN,11926.700195,NaN
2011-12-27,NaN,NaN,NaN,NaN,11926.700195,NaN
2011-12-28,11893.400391,11917.000000,11709.799805,11728.400391,11728.400391,98273900.0
2011-12-29,11703.200195,11842.099609,11703.200195,11841.700195,11841.700195,95627700.0


In [3]:
# 2
# 3-Day rolling average
tsx['MA3'] = tsx['Adj Close'].rolling(3).mean()

# Resample daily to weekly data with the avg of 3-day MA for the week (week ends on fri)
weekly = tsx['MA3'].resample('W-FRI').mean()

In [7]:
# 3
# make a series of log returns for the weekly series of 3D MA
weekly_log = np.log(weekly/weekly.shift(1)) # = log(current price/previous price)
weekly_log = weekly_log.dropna() # drop first row with NA
weekly_log

2007-01-12   -0.011207
2007-01-19    0.014978
2007-01-26    0.011994
2007-02-02    0.012762
2007-02-09    0.010324
                ...   
2011-12-02    0.010503
2011-12-09    0.021235
2011-12-16   -0.027699
2011-12-23   -0.004623
2011-12-30    0.014868
Freq: W-FRI, Name: MA3, Length: 260, dtype: float64

In [5]:
# 4 
mean_wlnrt = weekly_log.mean() 
std_wlnrt = weekly_log.std()

print(f"Mean of Log Returns: {100*mean_wlnrt:.2f}%")
print(f"Std Deviation of Log Returns: {100*std_wlnrt:.2f}%")

Mean of Log Returns: -0.02%
Std Deviation of Log Returns: 2.35%


In [6]:
# 5 
annual_mean = mean_wlnrt*52
annual_std = std_wlnrt*np.sqrt(52)

print(f"Annualized Mean (Drift): {100*annual_mean:.2f}%")
print(f"Annualized Std Deviation (Volatility): {100*annual_std:.2f}%")

Annualized Mean (Drift): -1.27%
Annualized Std Deviation (Volatility): 16.92%
